loading the data

In [1]:
import langchain
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader(r"C:\Users\Chandana c s\Desktop\constitution\data_ingestion\Indian constitution.pdf")
docs=loader.load()

splitting the data

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=splitter.split_documents(docs)

In [3]:
len(docs)

1120

embedding


In [4]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os


c:\Users\Chandana c s\Desktop\constitution\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))


In [6]:
def download_embeddings():
    embeddings = GoogleGenerativeAIEmbeddings(model ="models/embedding-001")
    return embeddings

# Get embeddings
embeddings = download_embeddings()

In [7]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 768


store the data in a vectorestore


In [9]:
from langchain_community.vectorstores import FAISS
embeddings = GoogleGenerativeAIEmbeddings(model ="models/embedding-001")
vectordb= FAISS.from_documents(docs,embedding=embeddings)

In [16]:
vectordb.save_local("faiss_index")


In [17]:
docsearch = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [18]:
docsearch

In [19]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [20]:
retrieved_docs = retriever.invoke("what are fundamental rights")

In [21]:
retrieved_docs

[Document(metadata={'source': 'C:\\Users\\Chandana c s\\Desktop\\constitution\\data_ingestion\\Indian constitution.pdf', 'page': 40}, page_content='THE CONSTITUTION OF  INDIA(Part III.—Fundamental Rights)10(e) to reside and settle in any part of the territory of India; 1[and]2[(f)*                      *                      *                    *                     *] (g) to practise any profession, or to carry on any occupation, trade or business. 3[(2) Nothing in sub-clause (a) of clause (1) shall affect the operation of any existing law, or prevent the State from making any law, in so far as such law imposes reasonable restrictions on the exercise of the right conferred by the said sub-clause in the interests of 4[the sovereignty and integrity of India], the security of the State, friendly relations with foreign States, public order, decency or morality, or in relation to contempt of court, defamation or incitement to an offence.](3) Nothing in sub-clause (b) of the said clause sh

In [23]:
model = ChatGoogleGenerativeAI(model = "gemini-1.5-flash")

In [24]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [25]:
question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [26]:
response = rag_chain.invoke({"input": "what is the meaning of secularism"})
print(response["answer"])

Based on the provided text, secularism is not explicitly defined.  The excerpts discuss religious freedom and the right to establish religious institutions, implying a separation of religious and state affairs, but don't offer a definition of secularism itself.



In [27]:
response = rag_chain.invoke({"input": "What are fundamental rights"})
print(response["answer"])

Based on the provided text, fundamental rights are detailed in Part III of the Indian Constitution.  They include rights to equality, freedom, against exploitation, freedom of religion, and more,  but the exact specifics are not fully described in this excerpt.  The text also mentions that laws inconsistent with these rights are void.

